# Informe Final de la Prueba Técnica 1

**Autor:** Isabel Castrillón Acosta

**Fecha:** 25-06-2025

**Descripción del Proyecto:**


# 1. Explicación del Caso

- Empresa de construcción necesita estimar el costo de un proyecto de 36 meses de duración (tres años).

- Se debe estimar el costo de los equipos, con base en el cosato de las materias primas que lo componen (X, Y, Z). 

**Detalles de Equipos**

- Equipo 1: $0.2 * PrecioX + 0.8 * PrecioY $

- Equipo 2: $\frac{(PrecioX + PrecioY + PrecioZ)}{3} $

**Datos**

- Todos los dataframes tienen rangos de tiempos diferentes en la columna "Fecha" y una columna de "Precio".
- Se cuentan con tres dataframes:
  - X: contiene el costo de las materias primas X.
  - Y: contiene el costo de las materias primas Y.
  - Z: contiene el costo de las materias primas Z.


**Beneficios esperados**
- Presición en la estimación de costos, lo que permitirá una mejor planificación financiera del proceso.
- Optimización de recursos al seleccionar proveedores con la mejor relación costo-beneficio. 






# 2. Supuestos

# 3. Formas de resolver el caso y la opcion elegida

- Ya que cada dataframe tiene una cantidad de filas diferente y un rango de tiempo distinto, se puede hacer una regresion lineal para cada materia prima y luego hacer predicciones a futuro para las mismas fechas de las tres materias primas.
- Una vez obtenidas las predicciones, se pueden aplicar las formulas de los equipos, para calcular los costos a futuro de ambos equipos. 

# 4. Resultados del análisis de los datos y los modelos

## 4.1 Librerias

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np


## 4.2 Cargar los dataframes



In [2]:
x = pd.read_csv('../Datos/X_cleaned.csv')
y = pd.read_csv('../Datos/Y_cleaned.csv')
z = pd.read_csv('../Datos/Z_cleaned.csv')

In [3]:
x.head(), y.head(), z.head()

(         Date  Price
 0  2024-04-04  89.18
 1  2024-04-03  89.35
 2  2024-04-02  88.92
 3  2024-04-01  87.42
 4  2024-03-28  87.48,
          Date   Price
 0  2023-09-12  547.33
 1  2023-09-11  546.00
 2  2023-09-08  545.00
 3  2023-09-07  550.00
 4  2023-09-06  552.50,
          Date    Price
 0  2010-01-01  2225.25
 1  2010-01-04  2225.25
 2  2010-01-05  2246.50
 3  2010-01-06  2302.50
 4  2010-01-07  2306.50)

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

# Ejemplo con materia prima X
df_x['timestamp'] = pd.to_datetime(df_x['fecha']).map(pd.Timestamp.toordinal)
X = df_x[['timestamp']]
y = df_x['precio']

model_x = LinearRegression().fit(X, y)

# Predicción a una fecha futura
futuro = pd.to_datetime('2026-01-01').toordinal()
precio_x_futuro = model_x.predict([[futuro]])[0]

# 5. Futuras mejoras

# 6. Apreciaciones y comentarios del caso (opcional)